# Example of generating QAs for an ML book (using self-instruct)
Source: https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html

### Before running the code

You will need to have the following packages installed:
```
pip install langchain pandas unstructured
```

Also, make sure you have a .env file with your OpenAI API key in the root directory of this project.
```
OPENAI_API_KEY=YOUR_API_KEY
```

## Load packages

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from uniflow.client import Client
from uniflow.config import OpenAIJsonConfig
from langchain.document_loaders import UnstructuredHTMLLoader
from uniflow.schema import Context, GuidedPrompt

load_dotenv()

/Users/joseortiz/anaconda3/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Prepare the input data

Uncomment any of the html files below as the sample file to build the self-instruct flow.

In [3]:
#html_file = "do_things_that_dont_scale.html" #from http://paulgraham.com/ds.html
#html_file = "makers_schedule_managers_schedule.html" #from http://www.paulgraham.com/makersschedule.html
#html_file = "life_is_short.html" #http://www.paulgraham.com/vb.html
html_file = "22.11_information-theory.html"

Set current directory and input data directory.

In [4]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", html_file)

In [5]:
loader = UnstructuredHTMLLoader(input_file)
pages = loader.load_and_split()

## Prepare input dataset

In [6]:
guided_prompt = GuidedPrompt(
        instruction="Generate one question and its corresponding answer based on context. Following the format of the examples below to include the same context, question, and answer in the response.",
        examples=[
            Context(
                context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
                question="Who published A Mathematical Theory of Communication in 1948?",
                answer="Claude E. Shannon.",
            )
        ]
)

data = [ Context(context=p) for p in pages[2].page_content.split("\n\n") if len(p) > 200]

In [7]:
data = data[-3:]
data


[Context(context='Any notion of information we develop must conform to this intuition.\nIndeed, in the next sections we will learn how to compute that these\nevents have \\(0\\textrm{ bits}\\), \\(2\\textrm{ bits}\\),\n\\(~5.7\\textrm{ bits}\\), and \\(~225.6\\textrm{ bits}\\) of\ninformation respectively.'),
 Context(context='If we read through these thought experiments, we see a natural idea. As\na starting point, rather than caring about the knowledge, we may build\noff the idea that information represents the degree of surprise or the\nabstract possibility of the event. For example, if we want to describe\nan unusual event, we need a lot information. For a common event, we may\nnot need much information.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.')]

## Run ModelFlow

In [8]:
config = OpenAIJsonConfig(guided_prompt_template=guided_prompt)
client = Client(config)

In [9]:
output = client.run(data)

100%|██████████| 3/3 [00:05<00:00,  1.92s/it]


In [10]:
output

[{'output': [{'response': [{'context': 'Any notion of information we develop must conform to this intuition. Indeed, in the next sections we will learn how to compute that these events have \\(0\\textrm{ bits}\\), \\(2\\textrm{ bits}\\), \\(~5.7\\textrm{ bits}\\), and \\(~225.6\\textrm{ bits}\\) of information respectively.',
      'question': 'What will we learn to compute in the next sections?',
      'answer': 'The amount of information that different events have.'}],
    'error': 'No errors.'}],
  'root': <uniflow.node.node.Node at 0x176ad4af0>},
 {'output': [{'response': [{'context': 'If we read through these thought experiments, we see a natural idea. As\na starting point, rather than caring about the knowledge, we may build\noff the idea that information represents the degree of surprise or the\nabstract possibility of the event. For example, if we want to describe\nan unusual event, we need a lot of information. For a common event, we may\nnot need much information.',
      'qu

## Format result into pandas table

In [12]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

for item in output:
    for i in item['output']:
        for response in i['response']:
            contexts.append(response['context'])
            questions.append(response['question'])
            answers.append(response['answer'])

df = pd.DataFrame({
    'context': contexts,
    'question': questions,
    'answer': answers
})

# Set display options
pd.set_option('display.max_colwidth', None)  # or use a specific width like 50
pd.set_option('display.width', 1000)

df

,context,question,answer
0,"Any notion of information we develop must conform to this intuition. Indeed, in the next sections we will learn how to compute that these events have \(0\textrm{ bits}\), \(2\textrm{ bits}\), \(~5.7\textrm{ bits}\), and \(~225.6\textrm{ bits}\) of information respectively.",What will we learn to compute in the next sections?,The amount of information that different events have.
1,"If we read through these thought experiments, we see a natural idea. As\na starting point, rather than caring about the knowledge, we may build\noff the idea that information represents the degree of surprise or the\nabstract possibility of the event. For example, if we want to describe\nan unusual event, we need a lot of information. For a common event, we may\nnot need much information.",What does information represent according to the natural idea presented in the thought experiments?,Information represents the degree of surprise or the abstract possibility of the event.
2,"In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",What did Shannon introduce for the first time in his 1948 article?,Shannon introduced the concept of information entropy for the first time.
